In [1]:
# !pip install selenium
# !pip install underthesea # work_tokenize, sentiment
# ! pip install transformers

In [3]:
import urllib.request
from bs4 import BeautifulSoup # parse html
import re #regex
import csv
import os
import json
import pandas as pd
import urllib.request
import joblib
from underthesea import word_tokenize #word_tokenize of lines
import numpy as np
import transformers as ppb # load model BERT
from transformers import BertModel, BertTokenizer
import torch
from sklearn.model_selection import train_test_split
# scrap comment = selenium
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
# import requests

# Craw comment of product tiki, lazada

In [74]:


def load_url_selenium_lazada(url):
    # Selenium
    driver=webdriver.Chrome(executable_path="C:/Program Files/webdriver/chromedriver.exe")
    print("Loading url=", url)
    driver.get(url)
    list_review = []
    # just craw 10 page
    # x=0
    # while x<10:
    #     # try:
    #     #     #Get the review details here
    #     #     WebDriverWait(driver,5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,".mod-reviews")))
    #     # except:
    #     #     print('No comment')
    #     #     break
    #     # x +=1
    #     product_reviews = driver.find_elements_by_css_selector("[class='item']")
    #     # Get product review
    #     for product in product_reviews:
    #         review = product.find_element_by_css_selector("[class='content']").text
    #         if (review != "" or review.strip()):
    #             print(review, "\n")
    #             list_review.append(review)
    #     #Check for button next-pagination-item have disable attribute then jump from loop else click on the next button
    #     if len(driver.find_elements_by_css_selector("button.next-pagination-item.next[disabled]"))>0:
    #         break
    #     else:
    #         button_next=WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button.next-pagination-item.next")))
    #         driver.execute_script("arguments[0].click();", button_next)
    #         print("next page")
    #         time.sleep(2)
    #         x +=1
    driver.close()
    return list_review

def load_url_selenium_tiki(url):
    driver=webdriver.Chrome(executable_path="C:/Program Files/webdriver/chromedriver.exe")
    print("Loading url=", url)
    driver.get(url)
    list_review = []
    # just craw 10 page
    x=0
    while x<10:
        try:
          
            #Get the review details here
            WebDriverWait(driver,5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,"div.review-comment")))
        except :
            print('No comment!')
            break
        product_reviews = driver.find_elements_by_css_selector("[class='review-comment']")
        # Get product review
        for product in product_reviews:
            review = product.find_element_by_css_selector("[class='review-comment__content']").text
            if (review != "" or review.strip()):
                print(review, "\n")
                list_review.append(review)
        #Check for button next-pagination-item have disable attribute then jump from loop else click on the next button
        try:
            #driver.find_element_by_xpath("//li[@class='btn next']/a").click()
            button_next=WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[class = 'btn next']")))
            driver.execute_script("arguments[0].click();", button_next)
            print("next page")
            time.sleep(2)
            x +=1
        except (TimeoutException, WebDriverException) as e:
            print('Load several page!')
            break
    driver.close()
    return list_review



# Standard data, tokenizer

In [75]:

def standardize_data(row):
    # remove stopword
    # Remove . ? , at index final
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Remove all . , " ... in sentences
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")

    row = row.strip()
    return row

# Tokenizer
def tokenizer(row):
    return word_tokenize(row, format="text")

def analyze(result):
    bad = np.count_nonzero(result)
    good = len(result) - bad
    print("No of bad and neutral comments = ", bad)
    print("No of good comments = ", good)

    if good>bad:
        return "Good! You can buy it!"
    else:
        return "Bad! Please check it carefully!"


# Processing data

In [76]:
def processing_data(data):
    # 1. Standardize data
    data_frame = pd.DataFrame(data)
    print('data frame:', data_frame)
    data_frame[0] = data_frame[0].apply(standardize_data)

    # 2. Tokenizer
    data_frame[0] = data_frame[0].apply(tokenizer)

    # 3. Embedding
    X_val = data_frame[0]
    return X_val


# Load Pretrain model BERT

In [77]:
def load_pretrainModel(data):
    
    '''
    Load pretrain model/ tokenizers
    Return : features
    '''
    model = BertModel.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    #encode lines
    tokenized = data.apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))

    # get lenght max of tokenized
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    print('max len:', max_len)

    # if lenght of tokenized not equal max_len , so padding value 0
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    print('padded:', padded[1])
    print('len padded:', padded.shape)

    #get attention mask ( 0: not has word, 1: has word)
    attention_mask = np.where(padded ==0, 0,1)
    print('attention mask:', attention_mask[1])

    # Convert input to tensor
    padded = torch.tensor(padded)
    attention_mask = torch.tensor(attention_mask)


    # Load model
    with torch.no_grad():
        last_hidden_states = model(padded, attention_mask =attention_mask)
    #     print('last hidden states:', last_hidden_states)

    features = last_hidden_states[0][:,0,:].numpy()
    print('features:', features)
    
    return features


# Predict

In [78]:
def predict(url):
    # 1. Load URL and print comments
    if url== "":
        url = "https://tiki.vn/dien-thoai-samsung-galaxy-m31-128gb-6gb-hang-chinh-hang-p58259141.html"
    data = load_url_selenium_lazada(url)
    # data = load_url_selenium_tiki(url)
    data = processing_data(data)
    features = load_pretrainModel(data)
    # 2. Load weights
    model = joblib.load('save_model.pkl')
    # 3. Result
    result = model.predict(features)
    print(result)
    print(analyze(result))

# predict(url ='https://tiki.vn/dien-thoai-samsung-galaxy-s20-plus-hang-chinh-hang-p48886153.html?src=search&2hi=0&keyword=s20&src=mega-menu')
predict(url = 'https://www.lazada.vn/products/op-lung-iphone-6-6s-plus-7-plus-8-plus-x-xr-xsmax-11-11-pro-11-promax-12-12-pro-12-promax-13-promax-trong-suot-chong-soc-i1277055288-s7560624279.html?search=1')

Loading url= https://www.lazada.vn/products/op-lung-iphone-6-6s-plus-7-plus-8-plus-x-xr-xsmax-11-11-pro-11-promax-12-12-pro-12-promax-13-promax-trong-suot-chong-soc-i1277055288-s7560624279.html?search=1


KeyboardInterrupt: 